<a href="https://colab.research.google.com/github/RhythmBear/100-DAYS-OF-CODE/blob/main/Kibo_ETL_TASK_Adepoju_Emmanuel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KIBO ETL Task BY ADEPOJU EMMANUEL


## Configuration
In This section i define everything needed to run the notebook smoothly. Which include;
- Importing the nessesary python Libraries
- Defining Connections to credentials for the service account.
- Connecting to the Google Api using oauth2client


In [50]:
!pip install pyairtable

In [51]:
# Import the nessesary Libraries needed to work with Google sheets api
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
import pandas as pd
import numpy as np
from google.colab import userdata

In [52]:
# Mount the Google drive.


drive.mount('/content/drive')

# For security reasons the path to the credentials.json file is stored as a
# secret in the google colab settings and accessed here.
credentials_path = userdata.get('credentials_path')
airtable_access_token = userdata.get('airtable_access_token')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
# Define the scope that the service account covers.
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

# Creating Credentials for a Service Account.
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)


## Extraction Stage
Now that everything needed to run our ETL is all setup. I am going to go ahead and access the data and display it using pandas.

In [54]:
# Authorize the client
client = gspread.authorize(creds)

# Open the Google Sheet using its URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1hl5oNQVdO-BN7je7cOKViiJ6E-rQ1DFxORhimOzCPEA/edit?usp=sharing'
sheet = client.open_by_url(sheet_url)

# Assign all the worksheets to different variables.
progress_sheet = sheet.get_worksheet(0)
project_sheet = sheet.get_worksheet(1)
final_sheet = sheet.get_worksheet(2)


In [55]:
# Load the data from the source into a pandas datafram
progress_df = pd.DataFrame(progress_sheet.get_all_records())
progress_df.head()

project_df = pd.DataFrame(project_sheet.get_all_records())
# project_df.info()

In [56]:
# progress_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 88 columns):
 #   Column                                                       Non-Null Count  Dtype 
---  ------                                                       --------------  ----- 
 0   Email (fake)                                                 81 non-null     object
 1   Assignment 4: Squad Info Bot Status                          81 non-null     object
 2   Assignment 4: Squad Info Bot Timestamp                       81 non-null     object
 3   Bonus: Quick Draw Project Status                             81 non-null     object
 4   Bonus: Quick Draw Project Timestamp                          81 non-null     object
 5   Final Project Status                                         81 non-null     object
 6   Final Project Timestamp                                      81 non-null     object
 7   hello-world Status                                           81 non-null     object
 8   he

In [58]:
# Create a Test Data frame and try the melt funtion on it.
test_prog_df = progress_df[:3]
test_prog_df

,Email (fake),Assignment 4: Squad Info Bot Status,Assignment 4: Squad Info Bot Timestamp,Bonus: Quick Draw Project Status,Bonus: Quick Draw Project Timestamp,Final Project Status,Final Project Timestamp,hello-world Status,hello-world Timestamp,hello-you Status,...,W4.2 - Mutating A List Status,W4.2 - Mutating A List Timestamp,W4.3 - Sum of Range Status,W4.3 - Sum of Range Timestamp,W4.4 Loop Debugging With Print Status,W4.4 Loop Debugging With Print Timestamp,W4.5 - Sum of Odd Numbers Status,W4.5 - Sum of Odd Numbers Timestamp,W4.6 - Interactive Minimum Status,W4.6 - Interactive Minimum Timestamp
0,bernier.victoria@hotmail.com,waiting for review,2022-06-03 22:58:49,waiting for review,2022-05-08 18:23:02,waiting for review,2022-06-03 20:18:21,waiting for review,2022-04-15 12:02:06,waiting for review,...,waiting for review,2022-05-26 21:55:08,in progress,2022-05-29 22:20:19,in progress,2022-05-29 22:50:53,in progress,2022-05-29 22:52:30,in progress,2022-05-29 22:53:10
1,delpha.doyle@dooley.com,waiting for review,2022-05-26 1:24:31,waiting for review,2022-05-08 17:50:12,waiting for review,2022-05-27 7:36:13,waiting for review,2022-04-20 2:48:19,waiting for review,...,waiting for review,2022-05-18 1:33:58,waiting for review,2022-05-24 12:23:43,waiting for review,2022-05-24 13:55:37,waiting for review,2022-05-24 12:29:43,waiting for review,2022-05-24 14:45:07
2,ayana76@gmail.com,waiting for review,2022-05-26 9:36:33,waiting for review,2022-05-12 18:40:29,waiting for review,2022-06-03 16:36:08,waiting for review,2022-04-20 6:34:13,waiting for review,...,waiting for review,2022-05-20 20:05:58,waiting for review,2022-05-22 5:06:38,waiting for review,2022-05-22 5:21:33,waiting for review,2022-05-22 8:26:37,waiting for review,2022-05-22 12:31:53


In [59]:
# Generate all the  Status and timestamp column names from the subjects dataframe
status_columns = [column['Projects'] + " Status" for ind, column in project_df.iterrows()]
timestamp_columns = [column['Projects'] + " Timestamp" for ind, column in project_df.iterrows()]

# Get the columns that have the test passed i/o stuff
scored_columns = [column for column in progress_df.columns if "I/O Tests Passed" in column]

print(len(status_columns))
print(len(timestamp_columns))
print(len(scored_columns))


38
38
11


## Testing the Transformation before applying it on the actual dataset.

## Define the Function to Transform the Data into the Required Format



In [60]:
# Define transformation.

def transform_pivoted_table_to_long_format(sample_dataframe):
  assert isinstance(sample_dataframe, pd.DataFrame), "sample_dataframe must be a pandas DataFrame"
  # assert all(status_columns in sample_dataframe.columns), "sample_dataframe must have all the status columns"
  # assert all(timestamp_columns in sample_dataframe.columns), "sample_dataframe must have all the timestamp columns"
  # assert all(scored_columns in sample_dataframe.columns), "sample_dataframe must have all the scored columns"

  #Create 3 different tables using the melt function.'
  # Melt the table with the columns that have the project status.
  status_unpivot_df = sample_dataframe.melt(
    id_vars="Email (fake)",
    value_vars=status_columns,
    var_name="Project",
    value_name="Status")

  # Remove the status from the project name
  status_unpivot_df['project'] = status_unpivot_df['Project'].str.split(' ').str[:-1].str.join(' ')

  # Melt the table with the Columns that have the timestamp.
  timestamp_unpivot_df = sample_dataframe.melt(
    id_vars="Email (fake)",
    value_vars=timestamp_columns,
    var_name="Project",
    value_name="Timestamp")

  # Remove the extra 'timestamp' at the end of the Project Name
  timestamp_unpivot_df['project'] = timestamp_unpivot_df['Project'].str.split(' ').str[:-1].str.join(' ')

  # Unpivot the scored columns
  scored_unpivot_columns = sample_dataframe.melt(
    id_vars="Email (fake)",
    value_vars=scored_columns,
    var_name="Project",
    value_name="points_earned")
  # Creat the max_points_obtainable column.
  scored_unpivot_columns['max_points_obtainable'] = scored_unpivot_columns['Project'].str.split(' ').str[-1].str.strip('()').astype(int)
  scored_unpivot_columns

  # Removing the extra identifier from the Project name (I/o test passed (x)) and creating a new column called  project with the transformed data
  scored_unpivot_columns['project'] = scored_unpivot_columns['Project'].str.split(' ').str[:-4].str.join(' ')
  scored_unpivot_columns


  ## Merge the 3 melted Tables into one table.
  joined_time_and_status_df = pd.merge(status_unpivot_df, timestamp_unpivot_df, on=['Email (fake)', 'project'], how='inner')
  final_df = pd.merge(joined_time_and_status_df, scored_unpivot_columns, on=['Email (fake)', 'project'], how='left')

  result_df = final_df[['Email (fake)', 'project', 'Status', 'Timestamp', 'points_earned', 'max_points_obtainable']]

  # Calculate the Missing fields and replace the NaN values with 0
  # result_df['Timestamp'] = result_df['Timestamp'].fillna(None)
  result_df['points_earned'] = result_df['points_earned'].fillna(0)
  result_df['max_points_obtainable'] = result_df['max_points_obtainable'].fillna(0)
  result_df['Assignment Grade'] = round((result_df['points_earned'] / result_df['max_points_obtainable']) * 100, 2)
  result_df['Assignment Grade'] = result_df['Assignment Grade'].fillna(0)

  # Join the result df with the projects sheet to get the grade weight for the project.
  new_joined_df = pd.merge(result_df, project_df, left_on='project', right_on='Projects', how='left')
  new_joined_df['Gradeweight'] = new_joined_df['Gradeweight'] / 100
  new_joined_df.drop(columns='Projects', inplace=True)

  # Rename COumns.
  new_joined_df.rename(columns={'Email (fake)': 'Email',
                                'points_earned': 'Points Earned',
                                'max_points_obtainable': 'Max Points Obtainable',
                                'project': 'Project'}, inplace=True)

  return new_joined_df



In [61]:
# Transform the progress_df
# test_df = progress_df[:1]
result_df = transform_pivoted_table_to_long_format(progress_df)
result_df

<ipython-input-60-f65ff1a622fe>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['points_earned'] = result_df['points_earned'].fillna(0)
<ipython-input-60-f65ff1a622fe>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['max_points_obtainable'] = result_df['max_points_obtainable'].fillna(0)
<ipython-input-60-f65ff1a622fe>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Email,Project,Status,Timestamp,Points Earned,Max Points Obtainable,Assignment Grade,Gradeweight
0,bernier.victoria@hotmail.com,hello-world,waiting for review,2022-04-15 12:02:06,0.0,0.0,0.0,0.01
1,delpha.doyle@dooley.com,hello-world,waiting for review,2022-04-20 2:48:19,0.0,0.0,0.0,0.01
2,ayana76@gmail.com,hello-world,waiting for review,2022-04-20 6:34:13,0.0,0.0,0.0,0.01
3,ihaag@beer.org,hello-world,waiting for review,2022-04-18 18:37:38,0.0,0.0,0.0,0.01
4,conroy.dahlia@gmail.com,hello-world,waiting for review,2022-04-19 3:16:13,0.0,0.0,0.0,0.01
...,...,...,...,...,...,...,...,...
3073,fbrekke@gmail.com,Print multiples,waiting for review,2022-05-11 8:42:03,0.0,0.0,0.0,0.01
3074,mattie.turcotte@yahoo.com,Print multiples,waiting for review,2022-05-15 21:59:41,0.0,0.0,0.0,0.01
3075,jdamore@rempel.info,Print multiples,waiting for review,2022-05-07 5:05:54,0.0,0.0,0.0,0.01
3076,fisher.lexus@monahan.com,Print multiples,waiting for review,2022-05-07 19:46:55,0.0,0.0,0.0,0.01


## Loading Stage.
Here I connect to the airtable API using the Airtable Access Token and batch load the transformed data into the database.


In [62]:
# Load data into Airtable.
from pyairtable import Api

air_client = Api(airtable_access_token)
student_progress_table = air_client.table(base_id='appNxMw4PQq2fW2vw', table_name='Student Progress')
# print(student_progress_table.schema())

id='tbli6VmVYOsmo3bxF' name='Student Progress' primary_field_id='fld6DUhhC3mxfiK2k' description=None fields=[AutoNumberFieldSchema(type='autoNumber', id='fld6DUhhC3mxfiK2k', name='ID', description=None), EmailFieldSchema(type='email', id='flddHfiISQGQrzL9m', name='Email', description=None), SingleLineTextFieldSchema(type='singleLineText', id='fldazb6kEbRHn20Um', name='Status', description=None), SingleLineTextFieldSchema(type='singleLineText', id='fldzqel1pgesEVQx2', name='Project', description=None), DateTimeFieldSchema(type='dateTime', options=DateTimeFieldOptions(time_zone='client', date_format=DateFormat(format='l', name='local'), time_format=TimeFormat(format='h:mma', name='12hour')), id='fldLif9mkOyWGVCWL', name='Timestamp', description=None), NumberFieldSchema(type='number', options=NumberFieldOptions(precision=2), id='fld7YypJbFtIISKYk', name='Max Points Obtainable', description=None), NumberFieldSchema(type='number', options=NumberFieldOptions(precision=2), id='fldPsvwdwmF5NUp

In [63]:
# Handle the null values in the Timestamp column
all_records = result_df.to_dict('records')

for record in all_records:
  if record['Timestamp'] == '':
    record['Timestamp'] = None
# Get the Dataframe in a structure that can be worked with .
result = student_progress_table.batch_create(all_records)
print(len(result))

3078
